#### Install the required dependencies

In [1]:
pip install transformers datasets evaluate rouge_score

Note: you may need to restart the kernel to use updated packages.


#### Load the dataset

In [2]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

/home/jovyan/Works/Practice/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
billsum = billsum.train_test_split(test_size=0.2)

In [4]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 17204 of the Business and Professions Code is amended to read:\n17204.\nActions for Injunctions by Attorney General, District Attorney, County Counsel, and City Attorneys\nActions for relief pursuant to this chapter shall be prosecuted exclusively in a court of competent jurisdiction by the Attorney General or a district attorney or by a county counsel authorized by agreement with the district attorney in actions involving violation of a county ordinance, or by a city attorney of a city having a population in excess of\n750,000,\n250,000,\nor by a city attorney in a city and county or, with the consent of the district attorney, by a city prosecutor in a city having a full-time city prosecutor in the name of the people of the State of California upon their own complaint or upon the complaint of a board, officer, person, corporation, or association, or by a person who has suffered injury in fact

In [6]:
print(billsum["train"][0]['text'])

The people of the State of California do enact as follows:


SECTION 1.
Section 17204 of the Business and Professions Code is amended to read:
17204.
Actions for Injunctions by Attorney General, District Attorney, County Counsel, and City Attorneys
Actions for relief pursuant to this chapter shall be prosecuted exclusively in a court of competent jurisdiction by the Attorney General or a district attorney or by a county counsel authorized by agreement with the district attorney in actions involving violation of a county ordinance, or by a city attorney of a city having a population in excess of
750,000,
250,000,
or by a city attorney in a city and county or, with the consent of the district attorney, by a city prosecutor in a city having a full-time city prosecutor in the name of the people of the State of California upon their own complaint or upon the complaint of a board, officer, person, corporation, or association, or by a person who has suffered injury in fact and has lost money 

In [7]:
print(billsum["train"][0]['summary'])

Existing law defines unfair competition to include an unlawful, unfair, or fraudulent business act or practice, unfair, deceptive, untrue, or misleading advertising, and any false representations to the public. Existing law, as amended by Proposition 64 at the November 2, 2004, statewide general election, authorizes an action for relief from this prohibited conduct to be brought by, among others, a person who has suffered injury in fact and has lost money or property as a result of the unfair competition. Existing law also authorizes an action for relief from this prohibited conduct and for civil penalties to be brought by a city attorney of a city having a population in excess of 750,000.
This bill would expand this authorization to allow actions for relief and civil penalties by city attorneys of cities having a population in excess of 250,000, as provided.
The Subdivision Map Act and local ordinances authorize or require, under specified circumstances, the furnishing of specified ty

#### Load the model from the Hugging Face Hub

In [8]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#### Prepare the model training inputs

In [9]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map: 100%|██████████| 248/248 [00:00<00:00, 686.37 examples/s]


#### Data collator makses sure that the inputs are of the same size

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

#### Evalulation metrics

In [12]:
import evaluate

rouge = evaluate.load("rouge")

In [13]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

### Train the model

In [14]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_billsum_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
    # push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.775260,0.133700,0.043400,0.113100,0.113100,20.000000
2,No log,2.554997,0.144100,0.052700,0.119600,0.119500,20.000000
3,No log,2.461733,0.169100,0.068200,0.140000,0.139500,20.000000
4,No log,2.411007,0.196000,0.093200,0.164000,0.163800,20.000000
5,No log,2.372820,0.201400,0.097800,0.168500,0.168000,20.000000
6,No log,2.347858,0.200100,0.095800,0.167500,0.167100,20.000000
7,No log,2.323982,0.202400,0.098700,0.169500,0.169100,20.000000
8,No log,2.306109,0.202600,0.098100,0.169300,0.168800,20.000000
9,2.718500,2.292192,0.202700,0.098800,0.169200,0.168900,20.000000
10,2.718500,2.278517,0.203000,0.099000,0.170800,0.170400,20.000000


TrainOutput(global_step=1240, training_loss=2.5090435889459424, metrics={'train_runtime': 194.4302, 'train_samples_per_second': 101.733, 'train_steps_per_second': 6.378, 'total_flos': 5354121666232320.0, 'train_loss': 2.5090435889459424, 'epoch': 20.0})

### Save the model and tokenizer

In [16]:
# Save the model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/spiece.model',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

#### Inference with the fine-tuned model

In [17]:
text = """summarize: Artificial intelligence is transforming financial investment decision-making
frameworks, with deep reinforcement learning demonstrating substantial
potential in robo-advisory applications. This paper addresses the limitations
of traditional portfolio optimization methods in dynamic asset weight
adjustment through the development of a deep reinforcement learning-based
dynamic optimization model grounded in practical trading processes. The
research advances two key innovations: first, the introduction of a novel
Sharpe ratio reward function engineered for Actor-Critic deep reinforcement
learning algorithms, which ensures stable convergence during training while
consistently achieving positive average Sharpe ratios; second, the development
of an innovative comprehensive approach to portfolio optimization utilizing
deep reinforcement learning, which significantly enhances model optimization
capability through the integration of random sampling strategies during
training with image-based deep neural network architectures for
multi-dimensional financial time series data processing, average Sharpe ratio
reward functions, and deep reinforcement learning algorithms. The empirical
analysis validates the model using randomly selected constituent stocks from
the CSI 300 Index, benchmarking against established financial econometric
optimization models. Backtesting results demonstrate the model's efficacy in
optimizing portfolio allocation and mitigating investment risk, yielding
superior comprehensive performance metrics."""

In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")
inputs = tokenizer(text, return_tensors="pt").input_ids


In [19]:
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)
outputs[0]

tensor([    0,     3,     9,   126, 22130,    15,  5688,  9676,  1681,  1948,
         3737,    21,  1983,   127,    18,   254, 13224,    75,  1659, 28050,
         1036, 16783,     6,    84,   766,     7,  5711, 31098,   383,   761,
          298,  8182,     3,  9582,  1465,  1348, 22130,    15,  5688,     7,
            6,    11,  1659, 28050,  1036, 16783,     5,   100,  1040,  7181,
            8, 10005,    13,  1435,  4833, 11295,  2254,    16,  4896,  7000,
         1293, 13592,   190,     8,   606,    13,     3,     9,  3714, 22130,
           15,  5688,  9676,  1681,  1948,  3737,    21,  1983,   127,    18,
          254, 13224,    75,  1659, 28050,  1036, 16783,     6,    84,   766,
            7,  5711, 31098,   383,   761,   298,  8182,     3,  9582,  1465,
         1348])

In [20]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'a new Sharpe ratio reward function engineered for Actor-Critic deep reinforcement learning algorithms, which ensures stable convergence during training while consistently achieving positive average Sharpe ratios, and deep reinforcement learning algorithms. This paper addresses the limitations of traditional portfolio optimization methods in dynamic asset weight adjustment through the development of a novel Sharpe ratio reward function engineered for Actor-Critic deep reinforcement learning algorithms, which ensures stable convergence during training while consistently achieving positive average'